In [7]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import signal
import seaborn as sns
from scipy.io import wavfile
import pylab
import matplotlib.mlab as ml
from scipy.fftpack import fft
import pickle

%matplotlib tk

In [8]:
DEFAULT_FS = 44100
DEFAULT_WINDOW_SIZE = 4096
DEFAULT_OVERLAP_RATIO = 0.5
DEFAULT_FAN_VALUE = 15
DEFAULT_AMP_MIN = 10

def graph_spectrogram(sound_info, frame_rate):
    pylab.figure(num=None, figsize=(19, 12))
    pylab.subplot(111)
    pylab.specgram(sound_info, Fs=frame_rate)
    pylab.savefig('spectrogram.png')

"""
Function that converts a byte string into a numpy array
"""
def _wav2array(nchannels, sampwidth, data):
    num_samples, remainder = divmod(len(data), sampwidth * nchannels)
    if remainder > 0:
        raise ValueError('The length of data is not a multiple of '
                         'sampwidth * num_channels.')
    if sampwidth > 4:
        raise ValueError("sampwidth must not be greater than 4.")

    if sampwidth == 3:
        a = np.empty((num_samples, nchannels, 4), dtype=np.uint8)
        raw_bytes = np.fromstring(data, dtype=np.uint8)
        a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
        a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
        result = a.view('<i4').reshape(a.shape[:-1])
    else:
        dt_char = 'u' if sampwidth == 1 else 'i'
        a = np.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
        result = a.reshape(-1, nchannels)
    return result

"""
Function to convert stereo to mono
"""

def stereo2mono(audiodata, nchannels):
#     if nchannels==1:
#         return audiodata.astype(int)
    audiodata = audiodata.astype(float)
    d = audiodata.sum(axis=1) / 2
    return d.astype(int)

"""
Class containing details of the wav file that has been read.
Sample use:
    song_x = song("abc.wav")
"""
class song:
    def __init__(self, file, songid):
        wav = wave.open(file)
        self.song_id = songid
        self.title = file.split("/")[-1]
        self.rate = wav.getframerate()
        self.nchannels = wav.getnchannels()
        self.sampwidth = wav.getsampwidth()
        self.nframes = wav.getnframes()
        self.data = wav.readframes(self.nframes)
        self.array = stereo2mono(_wav2array(self.nchannels, self.sampwidth, self.data), self.nchannels)
        wav.close()
    def spectrogram(self):
        self.specgram, self.frequencies, self.times = ml.specgram(self.array, Fs=self.rate, NFFT = 4096, window = ml.window_hanning, noverlap = int(4096 * 0.5), mode='magnitude')
        self.specgram = 10*np.log10(self.specgram)
        self.specgram[self.specgram==-np.inf] = 0
#         self.specgram = (1/20)*(np.exp(self.specgram))
#         self.specgram[self.specgram<100000000000000000] = 100000000000000000
#         self.specgram[self.specgram>10000000000000000000] = 10000000000000000000
#         fig, ax = plt.subplots()
#         ax.imshow(self.specgram, aspect='auto')
#         ax.set_xlabel('Time')
#         ax.set_ylabel('Frequency')
#         ax.set_title("Spectrogram of "+self.title)
#         plt.gca().invert_yaxis()
#         plt.show()
    def find_key(self):
        self.spectrogram()
        all_times = self.specgram.transpose()
        #self.all_times = all_times
        bands = []
        count = 0
        for a in all_times:
            l = []
            x = max(a[0:10])
            l.append((x, [list(a[0:10]).index(x),self.times[count]]))
            x = max(a[10:20])
            l.append((x, [list(a[10:20]).index(x)+10,self.times[count]]))
            x = max(a[20:40])
            l.append((x, [list(a[20:40]).index(x)+20,self.times[count]]))
            x = max(a[40:80])
            l.append((x, [list(a[40:80]).index(x)+40,self.times[count]]))
            x = max(a[80:160])
            l.append((x, [list(a[80:160]).index(x)+80,self.times[count]]))
            x = max(a[160:510])
            l.append((x, [list(a[160:510]).index(x)+160,self.times[count]]))
            bands.append(l)
            count+=1
        l = []
        #print('length',len(bands))
        for a in bands:
            for b in a:
                l.append(b[0])
        #l has all the amplitudes in bands
        mean = .1*np.mean(l)
        new_bands = []
        for i in range(0, len(bands)):
            a = bands[i]
            m = [t[1] for t in a if t[0]>mean]
            if len(m)!=0:
                new_bands.append(m)
        self.bands = new_bands
    def cal_address(self):
        new_bands = []
        for ele in self.bands:
            for sub in ele:
                new_bands.append(sub)
        self.addresses = {}
        for i in range(3,len(new_bands)-4):
            anchor_point = new_bands[i-3]
            for ele in new_bands[i:i+5]:
                diff = float("%.2f"%(ele[1] - anchor_point[1]))
                val =  float("%.2f"%(anchor_point[1]))
                if (anchor_point[0],ele[0],diff) not in self.addresses.keys():
                    self.addresses[anchor_point[0],ele[0],diff] = []
                self.addresses[anchor_point[0],ele[0],diff].append([val,self.song_id])
        

In [9]:
song_dict = {}
hashtable = {}

# file = open("./allsongs", "r")#list of song files ----> use      ls >allsongs
# allsongs = file.readlines()
# allsongs = [a[:-1] for a in allsongs]

allsongs = ["Ricky Martin - Livin La Vida Loca.wav"]
num = 0

for a in allsongs:
    x = song("./Songs_Wav/"+a, num)
    song_dict[num] = a
    num+=1
    print(x.title)
    x.find_key()
    x.cal_address()
    for key in x.addresses.keys():
        if key not in hashtable.keys():
            hashtable[key] = []
        for b in x.addresses[key]:
            hashtable[key].append(b)
            

Ricky Martin - Livin La Vida Loca.wav


In [10]:
with open("hashtable.pkl", "wb") as output:
    pickle.dump(hashtable, output, -1)
with open("songid.pkl", "wb") as output:
    pickle.dump(song_dict, output, -1)

In [11]:
hashtable

{(0, 52, 0.0): [[0.05, 0],
  [0.09, 0],
  [0.14, 0],
  [0.19, 0],
  [222.03, 0],
  [222.08, 0],
  [222.12, 0],
  [222.17, 0],
  [222.21, 0],
  [222.26, 0],
  [222.35, 0],
  [222.4, 0],
  [222.49, 0],
  [222.59, 0],
  [222.63, 0],
  [222.68, 0],
  [222.73, 0],
  [222.77, 0],
  [222.82, 0]],
 (0, 80, 0.0): [[0.05, 0]],
 (0, 189, 0.0): [[0.05, 0]],
 (0, 0, 0.05): [[0.05, 0],
  [0.09, 0],
  [0.14, 0],
  [93.16, 0],
  [93.2, 0],
  [93.53, 0],
  [93.72, 0],
  [221.66, 0],
  [221.7, 0],
  [221.75, 0],
  [221.8, 0],
  [221.84, 0],
  [221.89, 0],
  [221.94, 0],
  [221.98, 0],
  [222.03, 0],
  [222.08, 0],
  [222.12, 0],
  [222.17, 0],
  [222.21, 0],
  [222.26, 0],
  [222.31, 0],
  [222.35, 0],
  [222.4, 0],
  [222.45, 0],
  [222.49, 0],
  [222.54, 0],
  [222.59, 0],
  [222.63, 0],
  [222.68, 0],
  [222.73, 0],
  [222.77, 0],
  [222.82, 0]],
 (0, 16, 0.05): [[0.05, 0], [92.93, 0], [93.16, 0], [93.72, 0], [222.59, 0]],
 (17, 80, 0.0): [[0.05, 0], [58.61, 0], [166.53, 0], [196.26, 0]],
 (17, 189, 